In [64]:
# Zener Diode Voltage Regulator

from scipy.integrate import quad
import matplotlib.pyplot as plt
import numpy as np


<h2>When the Load Resistor R_L Disconnected</h2>

<h3>Current Calculations</h3>
<ul>
    <li><p>When the load resistor is connected, the current through the load resistor should remain 10 mA,
    and the current through the Zener diode should ideally be 5 mA, but can conduct at a current as low as 1 mA and as high as 9 mA.</p></li>
    <li><p>When the load resistor is not connected, the 10 mA that would otherwise flow through the load resistor would increase the current through
    the Zener diode by 10 mA. Therefore, the maximum current through the Zener diode when the load resistor is not connected is 19 mA.</p></li>
    <li><p>A maximum current through the Zener diode of 19 mA is still less than the maximum specified in the data sheet of 20 mA.</p></li>
</ul>
<h3>Power Calculations</h3>
<ul>
    <li><p>Moreover, if the current through the Zener diode is 19 mA, the power dissipated by the Zener diode would then be calculated as P_max = I * V.</p></li>
    <li><p>P_max = 19 mA * 10 V = 190 mW. 190 mW is less than the maximum specfified power of 250 mW.</p></li>
</ul>

In [66]:
# Choosing R1 and C values from current calculations

# Constants
R_L = 1000  # [Ohms]
f = 60      # [Hz]
T = 1 / f   # [s]
V_D = 0.8   # [V] voltage of diodes in the wheatstone bridge

# Conditions for voltage of Zener diode at I_ZK = 5 mA
V_Z = 10
V_Z_min = 9.4
V_Z_max = 10.6

# Zener Diode Currents
I_Z0 = 5 * 10 ** -3             # Ideal given operating current:   5 mA
I_ZT2 = 1 * 10 ** -3            # Minimum given operating current: 1 mA 
I_ZT3 = 20 * 10 ** -3           # Maximum given operating current: 20 mA
deltaI = abs(I_Z0 - I_ZT2)      # Calculated deviation with s.f. +- 4 mA from ideal 
I_Z_Min = I_ZT2                 # Minimum calculated IZ current = 1 mA
I_Z_Max = I_Z0 + deltaI         # Maximum calculated w s.f. IZ = 9 mA

# Current Calculations
I_L = V_Z / R_L         # 10 mA
I_Z_ideal = I_Z0        # 5 mA
I_in = I_Z_ideal + I_L  # 15 mA

# Conditions for V_in [V]
V_in_pk = 15.4                   # V_in_pk: 15.4 V
delta_V_in = 1.136               # Calculated deviation deltaV_in: 1.136 V
V_in_max = 15.4 + delta_V_in     # V_in_max: 16.536 V
V_in_min = 15.4 - delta_V_in     # V_in_min: 14.234 V

R_Z = 20  # r_z = 20 Ohms @ I_Z = 5 mA

def calculate_R1(V_in, V_Z, I_in):
    return (V_in - V_Z) / I_in

def calculate_R_TH(R1, R_Z):
    return R1 + ((R_Z * R_L) / (R_Z + R_L))

# Method 1 from lab handout
def calculate_C_method1(R1):
    return (V_in_pk - 2*V_D - V_Z) / (240 * R1 * delta_V_in)

# Method 2 from lab handout
def calculate_C_from_integration(R_TH, R1):
    term1 = -1 / (120 * R_TH)
    term2 = 1 / np.log(1 - (((2 * delta_V_in) / (V_in_pk - 2*V_D - V_Z)) * (R1 / R_TH)))
    return term1 * term2

# Calculate R1, R_TH, C based V_in_min = 14.234 V, V_Z = 10 V, I_in = 15 mA
R1_small = calculate_R1(V_in_min, V_Z, I_in)
R_TH_small = calculate_R_TH(R1_small, R_Z)
C_1 = calculate_C_from_integration(R1_small, R_TH_small)
C_11 = calculate_C_method1(R1_small)

def I_C(t):
    return (V_in_pk - 2*V_D - V_Z) / R1_small * np.exp(-t / (R_TH_small * C_1))

# Calculate the definite integral of I_C(t) from 0 to 1/120 seconds
delta_Q, error = quad(I_C, 0, 1/120)
C_test = delta_Q / (2 * delta_V_in)
print(f"Calculations based on V_in = {V_in_min:0.3f} V, I_in = {I_in*10**3:0.2f} mA, V_Z = {V_Z}")
print(f"R1 = {R1_small:0.3f} Ohms\nR_TH = {R_TH_small:0.3f} Ohms\n")
print(f"Method 1: C = {C_11*10**6:.3f} uF")
print(f"Method 2 (my integral calcualtions): C = {C_1*10**6:.3f} uF")
print(f"Method 3: Calculting C from scipy C = {C_test*10**6:.3f} uF")

# Calculate R1, R_TH, C based on V_in_max = 16.536 V, V_Z = 10 V, I_in = 15 mA
R1_large = calculate_R1(V_in_max, V_Z, I_in)
R_TH_large = calculate_R_TH(R1_large, R_Z)
C_2 = calculate_C_from_integration(R1_large, R_TH_large)
print(f"\nCalculations based on V_in = {V_in_max:0.3f} V, I_in = {I_in*10**3:0.3f} mA, V_Z = {V_Z}")
print(f"R1 = {R1_large:0.3f} Ohms\nR_TH = {R_TH_large:0.3f} Ohms\nC = {C_2*10**6:0.3f} F")


Calculations based on V_in = 14.264 V, I_in = 15.00 mA, V_Z = 10
R1 = 284.267 Ohms
R_TH = 303.875 Ohms

Method 1: C = 49.031 uF
Method 2 (my integral calcualtions): C = 28.761 uF
Method 3: Calculting C from scipy C = 31.605 uF

Calculations based on V_in = 16.536 V, I_in = 15.000 mA, V_Z = 10
R1 = 435.733 Ohms
R_TH = 455.341 Ohms
C = 19.509 F


In [61]:
# Requirement Verifications for Currents
# Current across the Zener diode must be in range 1 mA < 9 mA when load resistor is connected

def calculate_current_across_capacitor(V_in_pk, V_D, V_Z, R1, tau, t):
    return ((V_in_pk - (2 * V_D) - V_Z) / R1) * np.exp(-t / tau)
    
# Using trapezoid method as approximation to calculate V_C(t) from I_C(t)
def calculate_voltage_across_capacitor(I_C, C, t):
    Q_cumulative = np.zeros(len(t))
    V_C = np.zeros(len(t))

    for i in range(1, len(t)):
        delta_t = t[i] - t[i - 1]
        avg_I = (I_C[i] - I_C[i - 1]) / 2
        delta_Q = avg_I * delta_t
        Q_cumulative[i] = Q_cumulative[i - 1] + delta_Q

    return Q_cumulative / C

# Calculate I_C(t), V_C(t) by calling function above
t = np.linspace(0, 1/120, 1000)  # Create array of 50 points from 0 to t = 1/120 s
tau1 = R_TH_large * C_1
tau2 = R_TH_small * C_2
exponent_1 = -t / (tau1)
exponent_2 = -t / (tau2)
I_C_1 = calculate_current_across_capacitor(V_in_pk, V_D, V_Z, R1_large, tau1, t)
I_C_2 = calculate_current_across_capacitor(V_in_pk, V_D, V_Z, R1_small, tau2, t)
V_C_1 = calculate_voltage_across_capacitor(I_C_1, C_1, t)
V_C_2 = calculate_voltage_across_capacitor(I_C_2, C_2, t)

print(V_C_1[len(V_C_1) - 1])

-0.0005953536495210839


In [ ]:
# Calculating Percent Errors from Simulated Values

# With R_L connected
V_out_avg = 10.0018       # [V]
V_out_pk2pk = 0.157164    # [V]
P_avg_zener = 0.0107589   # [W]
P_max_zener = 0.0166981   # [W]

# With R_L disconnected
V_out_avg_no_RL = 10.3087      # [V]
V_out_pk2pk_no_RL = 0.0197239  # [V]
P_avg_zener_no_RL = 0.107871   # [W]
P_max_zener_no_RL = 0.115496   # [W]